# **Online Retail Sales EDA**

In [ ]:
import pandas as pd

# Step 1: Load both sheets
file_path = "online_retail_II.xlsx"

df_2009 = pd.read_excel(file_path, sheet_name='Year 2009-2010', dtype={
    'Invoice': str,
    'StockCode': str,
    'Description': str,
    'Quantity': float,
    'Price': float,
    'Customer ID': float,
    'Country': str
})
df_2009['Year'] = '2009-2010'

df_2010 = pd.read_excel(file_path, sheet_name='Year 2010-2011', dtype={
    'Invoice': str,
    'StockCode': str,
    'Description': str,
    'Quantity': float,
    'Price': float,
    'Customer ID': float,
    'Country': str
})
df_2010['Year'] = '2010-2011'

# Combine both years
df = pd.concat([df_2009, df_2010], ignore_index=True)

# Step 2: Data Cleaning

# Drop rows with missing Customer ID or Description
df.dropna(subset=['Customer ID', 'Description'], inplace=True)

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Remove cancelled orders (Invoice starting with 'C')
df = df[~df['Invoice'].str.startswith('C')]

# Remove negative or zero Quantity and Price
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Add TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['Price']

# Final info
print("Cleaned Data Info:")
print(df.info())
print("\nSample Data:")
print(df.head())


In [2]:
# How many unique products?
unique_products = df['StockCode'].nunique()

# Total number of transactions
total_transactions = df['Invoice'].nunique()

# Total unique customers
unique_customers = df['Customer ID'].nunique()

# Countries involved
countries = df['Country'].nunique()
country_list = df['Country'].unique()

print("General Overview:")
print(f"Unique Products: {unique_products}")
print(f"Total Transactions: {total_transactions}")
print(f"Unique Customers: {unique_customers}")
print(f"Countries: {countries}")
print(f"Country List: {country_list}")


General Overview:
Unique Products: 4631
Total Transactions: 36969
Unique Customers: 5878
Countries: 41
Country List: ['United Kingdom' 'France' 'USA' 'Belgium' 'Australia' 'EIRE' 'Germany'
 'Portugal' 'Denmark' 'Netherlands' 'Poland' 'Channel Islands' 'Spain'
 'Cyprus' 'Greece' 'Norway' 'Austria' 'Sweden' 'United Arab Emirates'
 'Finland' 'Italy' 'Switzerland' 'Japan' 'Unspecified' 'Nigeria' 'Malta'
 'RSA' 'Singapore' 'Bahrain' 'Thailand' 'Israel' 'Lithuania' 'West Indies'
 'Korea' 'Brazil' 'Canada' 'Iceland' 'Lebanon' 'Saudi Arabia'
 'Czech Republic' 'European Community']


In [3]:
# Top 10 selling products by quantity
top_selling_products = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 Selling Products by Quantity:")
print(top_selling_products)

# Top 10 revenue-generating products
top_revenue_products = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 Revenue-Generating Products:")
print(top_revenue_products)

# Products with zero or negative prices
invalid_price_products = df[df['Price'] <= 0]['Description'].unique()
print("\nProducts with Zero or Negative Price:")
print(invalid_price_products)



Top 10 Selling Products by Quantity:
Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     108929.0
WHITE HANGING HEART T-LIGHT HOLDER     93520.0
PAPER CRAFT , LITTLE BIRDIE            80995.0
ASSORTED COLOUR BIRD ORNAMENT          79694.0
MEDIUM CERAMIC TOP STORAGE JAR         77916.0
JUMBO BAG RED RETROSPOT                75597.0
BROCADE RING PURSE                     71093.0
PACK OF 60 PINK PAISLEY CAKE CASES     55101.0
60 TEATIME FAIRY CAKE CASES            53320.0
PACK OF 72 RETROSPOT CAKE CASES        46046.0
Name: Quantity, dtype: float64

Top 10 Revenue-Generating Products:
Description
REGENCY CAKESTAND 3 TIER              285992.35
WHITE HANGING HEART T-LIGHT HOLDER    251731.26
PAPER CRAFT , LITTLE BIRDIE           168469.60
Manual                                151951.92
JUMBO BAG RED RETROSPOT               136684.79
ASSORTED COLOUR BIRD ORNAMENT         126704.06
POSTAGE                               126563.04
PARTY BUNTING                         103802.53
MEDIUM CERAMIC T

In [4]:
# Customers who bought the most products
top_customers_by_quantity = df.groupby('Customer ID')['Quantity'].sum().sort_values(ascending=False).head(10)
print("\nTop Customers by Quantity Purchased:")
print(top_customers_by_quantity)

# Distribution of purchases per customer
purchase_distribution = df.groupby('Customer ID')['Invoice'].nunique()

# Describe the distribution
print("\nCustomer Purchase Distribution Summary:")
print(purchase_distribution.describe())

# Loyal customers (most transactions)
loyal_customers = df.groupby('Customer ID')['Invoice'].nunique().sort_values(ascending=False).head(10)
print("\nTop 10 Loyal Customers (Most Transactions):")
print(loyal_customers)



Top Customers by Quantity Purchased:
Customer ID
14646.0    367193.0
13902.0    220600.0
13694.0    189205.0
18102.0    188340.0
14156.0    165873.0
14911.0    149949.0
17511.0    119656.0
16684.0    104810.0
14298.0    100272.0
12415.0     91447.0
Name: Quantity, dtype: float64

Customer Purchase Distribution Summary:
count    5878.000000
mean        6.289384
std        13.009406
min         1.000000
25%         1.000000
50%         3.000000
75%         7.000000
max       398.000000
Name: Invoice, dtype: float64

Top 10 Loyal Customers (Most Transactions):
Customer ID
14911.0    398
12748.0    336
17841.0    211
15311.0    208
13089.0    203
14606.0    192
14156.0    156
17850.0    155
14646.0    151
18102.0    145
Name: Invoice, dtype: int64


In [5]:
# Extract Month and Date
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
df['Weekday'] = df['InvoiceDate'].dt.day_name()

# Monthly sales trend
monthly_sales = df.groupby('InvoiceMonth')['TotalPrice'].sum()

# Daily pattern
daily_sales = df.groupby('InvoiceDate')['TotalPrice'].sum()

print("\nMonthly Sales Summary:")
print(monthly_sales)

print("\nSample of Daily Sales:")
print(daily_sales.head())



Monthly Sales Summary:
InvoiceMonth
2009-12     683504.010
2010-01     555802.672
2010-02     504558.956
2010-03     696978.471
2010-04     591982.002
2010-05     597833.380
2010-06     636371.130
2010-07     589736.170
2010-08     602224.600
2010-09     829013.951
2010-10    1033112.010
2010-11    1166460.022
2010-12     881079.100
2011-01     568101.310
2011-02     446084.920
2011-03     594081.760
2011-04     468374.331
2011-05     677355.150
2011-06     660046.050
2011-07     598962.901
2011-08     644051.040
2011-09     950690.202
2011-10    1035642.450
2011-11    1156205.610
2011-12     517208.440
Freq: M, Name: TotalPrice, dtype: float64

Sample of Daily Sales:
InvoiceDate
2009-12-01 07:45:00     505.30
2009-12-01 07:46:00     145.80
2009-12-01 09:06:00     630.33
2009-12-01 09:08:00     310.75
2009-12-01 09:24:00    2286.24
Name: TotalPrice, dtype: float64


In [6]:
# Orders per country
orders_by_country = df.groupby('Country')['Invoice'].nunique().sort_values(ascending=False)

# Revenue by country
revenue_by_country = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)

print("\nTop Countries by Number of Orders:")
print(orders_by_country.head(10))

print("\nTop Countries by Revenue:")
print(revenue_by_country.head(10))



Top Countries by Number of Orders:
Country
United Kingdom    33541
Germany             789
France              614
EIRE                567
Netherlands         228
Spain               154
Belgium             149
Sweden              104
Australia            95
Portugal             93
Name: Invoice, dtype: int64

Top Countries by Revenue:
Country
United Kingdom    1.466667e+07
EIRE              6.213043e+05
Netherlands       5.542307e+05
Germany           4.307038e+05
France            3.550594e+05
Australia         1.699006e+05
Spain             1.091272e+05
Switzerland       1.003653e+05
Sweden            9.151582e+04
Denmark           6.986219e+04
Name: TotalPrice, dtype: float64


**Step 4: RFM Segmentation**

In [7]:
import pandas as pd
import datetime as dt

# Set reference date as one day after the last invoice date
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Group by customer and calculate R, F, M
rfm = df.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'Invoice': 'nunique',                                      # Frequency
    'TotalPrice': 'sum'                                        # Monetary
}).reset_index()

# Rename columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Show RFM table
print("RFM Table Sample:")
print(rfm.head())


RFM Table Sample:
   CustomerID  Recency  Frequency  Monetary
0     12346.0      326         12  77556.46
1     12347.0        2          8   5633.32
2     12348.0       75          5   2019.40
3     12349.0       19          4   4428.69
4     12350.0      310          1    334.40


In [8]:
# Score Recency (lower is better)
def score_recency(r):
    if r <= 30:
        return 4
    elif r <= 90:
        return 3
    elif r <= 180:
        return 2
    else:
        return 1

# Score Frequency (higher is better)
def score_frequency(f):
    if f <= 2:
        return 1
    elif f <= 5:
        return 2
    elif f <= 10:
        return 3
    else:
        return 4

# Score Monetary (higher is better)
def score_monetary(m):
    if m <= 100:
        return 1
    elif m <= 500:
        return 2
    elif m <= 1000:
        return 3
    else:
        return 4

rfm['R'] = rfm['Recency'].apply(score_recency)
rfm['F'] = rfm['Frequency'].apply(score_frequency)
rfm['M'] = rfm['Monetary'].apply(score_monetary)

# Combine into RFM Score
rfm['RFM_Score'] = rfm['R'].astype(str) + rfm['F'].astype(str) + rfm['M'].astype(str)

# View top customers
print("\nTop RFM Customers:")
print(rfm.sort_values(by='RFM_Score', ascending=False).head())



Top RFM Customers:
      CustomerID  Recency  Frequency  Monetary  R  F  M RFM_Score
4672     17068.0       12         47   9153.83  4  4  4       444
5275     17682.0       10         11   4456.78  4  4  4       444
478      12826.0        3         12   2955.75  4  4  4       444
2191     14560.0        7         44   6114.03  4  4  4       444
1216     13577.0       25         11   3489.28  4  4  4       444


In [9]:
# Custom segmentation using RFM score
def segment_customer(row):
    score = row['RFM_Score']
    if score == '444':
        return 'Champions'
    elif row['R'] == 4:
        return 'Recent'
    elif row['F'] == 4:
        return 'Loyal'
    elif row['M'] == 4:
        return 'Big Spenders'
    elif score in ['111', '112', '121']:
        return 'At Risk'
    else:
        return 'Others'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# View segment counts
print(rfm['Segment'].value_counts())


Segment
Others          1388
Big Spenders    1289
At Risk         1267
Recent          1060
Champions        588
Loyal            286
Name: count, dtype: int64


In [ ]:
df.to_excel("cleaned_transactions.xlsx", index=False, engine='openpyxl')


In [ ]:
rfm.to_excel("rfm_segments.xlsx", index=False)


In [ ]:
monthly_sales = df.copy()
monthly_sales['Month'] = monthly_sales['InvoiceDate'].dt.to_period('M').astype(str)
monthly_summary = monthly_sales.groupby('Month').agg({
    'TotalPrice': 'sum',
    'Invoice': 'nunique',
    'Customer ID': 'nunique'
}).reset_index().rename(columns={
    'TotalPrice': 'Revenue',
    'Invoice': 'InvoiceCount',
    'Customer ID': 'CustomerCount'
})

monthly_summary.to_excel("monthly_sales_summary.xlsx", index=False)


In [ ]:
country_summary = df.groupby('Country').agg({
    'Invoice': 'nunique',
    'Customer ID': 'nunique',
    'TotalPrice': 'sum'
}).reset_index().rename(columns={
    'Invoice': 'InvoiceCount',
    'Customer ID': 'CustomerCount',
    'TotalPrice': 'Revenue'
})

country_summary.to_excel("country_sales_summary.xlsx", index=False)
